# Analyze Forced Photometry in ComCam Data

In [1]:
# %pip install lsdb dask nested-dask 

In [2]:
import lsdb
lsdb.__version__

'0.5.0'

In [3]:
from pathlib import Path

release = 'w_2025_07'
hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats/") / release
# list dir
print(list(map(str, hats_path.iterdir())))

comcam_obj = hats_path / "object"
comcam_src = hats_path / "forcedSource"

['/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaSource', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object_lc_x_ztf_dr22', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/source', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object_lc_x_ps1', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject_lc_x_ztf_dr22', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject_lc_x_ps1', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object_lc', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaForcedSource', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject_lc', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/forcedSource']


## Start Dask client

In [4]:
from dask.distributed import Client

# Start with a small client
client = Client(n_workers=1, memory_limit="16GB", threads_per_worker=1)
client

/sdf/data/rubin/user/wbeebe/miniforge3/envs/lsdb/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 9531 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:9531/status,
Dashboard: http://127.0.0.1:9531/status,Workers: 1
Total threads: 1,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:22271,Workers: 1
Dashboard: http://127.0.0.1:9531/status,Total threads: 1
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:21931,Total threads: 1
Dashboard: http://127.0.0.1:15891/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:8715,


# Load GAIA Data Around the COMCAM Field

In [5]:
source_lsdb_server =  "http://epyc.astro.washington.edu:43210/hats" # https://data.lsdb.io/hats is the alternative server

In [6]:
gaia_columns = [
    "solution_id",
    "designation",
    "source_id",
    "random_index",
    "ref_epoch",
    "ra",
    "ra_error", 
    "dec",
    "dec_error",
    "ruwe",
    "phot_variable_flag",
    "phot_g_mean_flux_over_error",
    "phot_bp_mean_flux_over_error",
    "phot_rp_mean_flux_over_error",
]
gaia_columns

['solution_id',
 'designation',
 'source_id',
 'random_index',
 'ref_epoch',
 'ra',
 'ra_error',
 'dec',
 'dec_error',
 'ruwe',
 'phot_variable_flag',
 'phot_g_mean_flux_over_error',
 'phot_bp_mean_flux_over_error',
 'phot_rp_mean_flux_over_error']

In [26]:
from upath import UPath
import hats
from lsdb.core.search import ConeSearch

catalogs_dir = UPath(source_lsdb_server)

# Gaia
gaia_path = catalogs_dir / "gaia_dr3" / "gaia"

# Define a 0.7 degree cone region of interest
# This includes the so called `Fornax dSph` field, one of the ComCam fields from
# https://community.lsst.org/t/locations-of-target-fields-observed-during-on-sky-commissioning-campaign-with-comcam/9609
cone_search = ConeSearch(ra=40, dec=-34.45, radius_arcsec=0.7 * 3600)


hats_gaia = hats.read_hats(gaia_path)

#gaia = lsdb.read_hats(gaia_path, columns=hats_gaia.schema.names, search_filter=cone_search) #filters=gaia_filters, )
gaia = lsdb.read_hats(gaia_path, columns=hats_gaia.schema.names, search_filter=cone_search) #filters=gaia_filters, )

In [27]:
# Filter column phot_variable_flag=CONSTANT
"""
gaia_filters = [
    #["phot_variable_flag", "=", "'CONSTANT'"],
    ["phot_g_mean_flux_over_error", ">", 20],
    ["phot_bp_mean_flux_over_error", ">", 20],
    ["phot_rp_mean_flux_over_error", ">", 20],
    #["ruwe", "<", 1.2],
]
"""
filtered_gaia = gaia.query("ruwe < 1.2").query("phot_variable_flag != 'VARIABLE'").query("phot_rp_mean_flux_over_error > 20")

In [28]:
filtered_gaia_compute = filtered_gaia.head(100)
filtered_gaia_compute

,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Dir,Npix
_healpix_29,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2525038339474834412,1636148068921376768,Gaia DR3 5050076667521318528,5050076667521318528,246159588,2016.0,40.08542,0.01182,-35.145041,0.015237,0.775229,0.020444,37.919006,5.926268,-5.902889,0.013726,0.525881,0.019373,-0.270601,0.173015,0.105584,-0.058346,-0.183463,-0.056218,0.11548,0.063183,-0.089452,-0.368243,660,0,657,3,-0.29032,668.10504,0.0,0.0,31,True,1.565983,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,76,31,0.028384,78,27,0,0.012573,89.401825,0,0,0.991461,0.16076,0.340183,0.025708,0.289681,-71.85327,67.1197,7.045306,-36.112274,False,642,18613.015381,3.997132,4656.5933,15.012825,74,10193.696413,11.188951,911.0503,15.317713,72,12098.90547,9.343641,1294.8812,14.54103,1.197689,0.0,6.0,0.0,2.0,0.0,0.776683,0.304888,0.471795,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NOT_AVAILABLE,238.72283,-65.450179,22.1061,-47.526707,False,False,0,True,False,False,False,False,True,True,False,0.0,0.0,0.999904,5830.5933,5794.376,5858.7935,4.4681,4.4228,4.5174,-0.5204,-0.5701,-0.4921,997.3952,947.5166,1051.0586,0

# Find GAIA objects that exhibit very little variability. 
For instance standard deviation less than 0.05 mag (for instance, flux_over_error >20, ruwe<1.2, phot_variable_flag=CONSTANT). 

## Loading & Nesting Forced Sources

In [80]:
# Load the Forced Source + MJD Table
from lsdb import read_hats


#BRIGHTEST_R_MAG = 21.5


obj = read_hats(
    comcam_obj,
    columns=["objectId", "coord_ra", "coord_dec"],
    #filters=[("r_psfMag", ">", BRIGHTEST_R_MAG)],
)
src_flat = read_hats(
    comcam_src,
    columns=[
        "objectId", 
        "coord_ra", "coord_dec",
        "band",
        "midpointMjdTai",
        "psfFlux", "psfFluxErr", "psfFlux_flag",
        "psfMag", "psfMagErr",
        "pixelFlags_suspect", "pixelFlags_saturated", "pixelFlags_cr", "pixelFlags_bad",
        "forcedSourceId",
        "detector",
        "visit",
    ],
)
src_nested = obj.join_nested(
    src_flat,
    nested_column_name="lc",
    left_on="objectId",
    right_on="objectId",
)
src_nested

/sdf/data/rubin/user/wbeebe/miniforge3/envs/lsdb/lib/python3.11/site-packages/lsdb/dask/join_catalog_data.py:322: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


,objectId,coord_ra,coord_dec,lc
npartitions=196,,,,
"Order: 5, Pixel: 32",int64[pyarrow],double[pyarrow],double[pyarrow],"nested<coord_ra: [double], coord_dec: [double]..."
"Order: 7, Pixel: 544",...,...,...,...
...,...,...,...,...
"Order: 6, Pixel: 35970",...,...,...,...
"Order: 6, Pixel: 35971",...,...,...,...


# Crossmatch ComCam Data and Gaia

In [29]:
comcam_gaia = src_nested.crossmatch(filtered_gaia)
comcam_gaia

/sdf/data/rubin/user/wbeebe/miniforge3/envs/lsdb/lib/python3.11/site-packages/lsdb/dask/crossmatch_catalog_data.py:105: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


,objectId_object,coord_ra_object,coord_dec_object,lc_object,solution_id_gaia,designation_gaia,source_id_gaia,random_index_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,parallax_error_gaia,parallax_over_error_gaia,pm_gaia,pmra_gaia,pmra_error_gaia,pmdec_gaia,pmdec_error_gaia,ra_dec_corr_gaia,ra_parallax_corr_gaia,ra_pmra_corr_gaia,ra_pmdec_corr_gaia,dec_parallax_corr_gaia,dec_pmra_corr_gaia,dec_pmdec_corr_gaia,parallax_pmra_corr_gaia,parallax_pmdec_corr_gaia,pmra_pmdec_corr_gaia,astrometric_n_obs_al_gaia,astrometric_n_obs_ac_gaia,astrometric_n_good_obs_al_gaia,astrometric_n_bad_obs_al_gaia,astrometric_gof_al_gaia,astrometric_chi2_al_gaia,astrometric_excess_noise_gaia,astrometric_excess_noise_sig_gaia,astrometric_params_solved_gaia,astrometric_primary_flag_gaia,nu_eff_used_in_astrometry_gaia,pseudocolour_gaia,pseudocolour_error_gaia,ra_pseudocolour_corr_gaia,dec_pseudocolour_corr_gaia,parallax_pseudocolour_corr_gaia,pmra_pseudocolour_corr_gaia,pmdec_pseudocolour_corr_gaia,astrometric_matched_transits_gaia,visibility_periods_used_gaia,astrometric_sigma5d_max_gaia,matched_transits_gaia,new_matched_transits_gaia,matched_transits_removed_gaia,ipd_gof_harmonic_amplitude_gaia,ipd_gof_harmonic_phase_gaia,ipd_frac_multi_peak_gaia,ipd_frac_odd_win_gaia,ruwe_gaia,scan_direction_strength_k1_gaia,scan_direction_strength_k2_gaia,scan_direction_strength_k3_gaia,scan_direction_strength_k4_gaia,scan_direction_mean_k1_gaia,scan_direction_mean_k2_gaia,scan_direction_mean_k3_gaia,scan_direction_mean_k4_gaia,duplicated_source_gaia,phot_g_n_obs_gaia,phot_g_mean_flux_gaia,phot_g_mean_flux_error_gaia,phot_g_mean_flux_over_error_gaia,phot_g_mean_mag_gaia,phot_bp_n_obs_gaia,phot_bp_mean_flux_gaia,phot_bp_mean_flux_error_gaia,phot_bp_mean_flux_over_error_gaia,phot_bp_mean_mag_gaia,phot_rp_n_obs_gaia,phot_rp_mean_flux_gaia,phot_rp_mean_flux_error_gaia,phot_rp_mean_flux_over_error_gaia,phot_rp_mean_mag_gaia,phot_bp_rp_excess_factor_gaia,phot_bp_n_contaminated_transits_gaia,phot_bp_n_blended_transits_gaia,phot_rp_n_contaminated_transits_gaia,phot_rp_n_blended_transits_gaia,phot_proc_mode_gaia,bp_rp_gaia,bp_g_gaia,g_rp_gaia,radial_velocity_gaia,radial_velocity_error_gaia,rv_method_used_gaia,rv_nb_transits_gaia,rv_nb_deblended_transits_gaia,rv_visibility_periods_used_gaia,rv_expected_sig_to_noise_gaia,rv_renormalised_gof_gaia,rv_chisq_pvalue_gaia,rv_time_duration_gaia,rv_amplitude_robust_gaia,rv_template_teff_gaia,rv_template_logg_gaia,rv_template_fe_h_gaia,rv_atm_param_origin_gaia,vbroad_gaia,vbroad_error_gaia,vbroad_nb_transits_gaia,grvs_mag_gaia,grvs_mag_error_gaia,grvs_mag_nb_transits_gaia,rvs_spec_sig_to_noise_gaia,phot_variable_flag_gaia,l_gaia,b_gaia,ecl_lon_gaia,ecl_lat_gaia,in_qso_candidates_gaia,in_galaxy_candidates_gaia,non_single_star_gaia,has_xp_continuous_gaia,has_xp_sampled_gaia,has_rvs_gaia,has_epoch_photometry_gaia,has_epoch_rv_gaia,has_mcmc_gspphot_gaia,has_mcmc_msc_gaia,in_andromeda_survey_gaia,classprob_dsc_combmod_quasar_gaia,classprob_dsc_combmod_galaxy_gaia,classprob_dsc_combmod_star_gaia,teff_gspphot_gaia,teff_gspphot_lower_gaia,teff_gspphot_upper_gaia,logg_gspphot_gaia,logg_gspphot_lower_gaia,logg_gspphot_upper_gaia,mh_gspphot_gaia,mh_gspphot_lower_gaia,mh_gspphot_upper_gaia,distance_gspphot_gaia,distance_gspphot_lower_gaia,distance_gspphot_upper_gaia,azero_gspphot_gaia,azero_gspphot_lower_gaia,azero_gspphot_upper_gaia,ag_gspphot_gaia,ag_gspphot_lower_gaia,ag_gspphot_upper_gaia,ebpminrp_gspphot_gaia,ebpminrp_gspphot_lower_gaia,ebpminrp_gspphot_upper_gaia,libname_gspphot_gaia,_dist_arcsec
npartitions=8,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 3, Pixel: 560",int64[pyarrow],double[pyarrow],double[pyarrow],"nested<coord_ra: [double], coord_dec: [double]...",int64[pyarrow],string[pyarrow],int64[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],dou

# Verify Crossmatch

Verify that the crossmatch is correct, for instance by confirming that objects that you have found in ComCam has roughly the same brightness in magnitudes as reported in GAIA. For example, compare Gaia RP mag to Rubin’s r mag, allowing up to 1-mag difference.

In [49]:
for col in comcam_gaia.columns:
    if "mag" in col:
        print(col)

phot_g_mean_mag_gaia
phot_bp_mean_mag_gaia
phot_rp_mean_mag_gaia
grvs_mag_gaia
grvs_mag_error_gaia
grvs_mag_nb_transits_gaia


In [31]:
comcam_gaia["lc_object"].nest.fields

['coord_ra',
 'coord_dec',
 'band',
 'midpointMjdTai',
 'psfFlux',
 'psfFluxErr',
 'psfFlux_flag',
 'psfMag',
 'psfMagErr',
 'pixelFlags_suspect',
 'pixelFlags_saturated',
 'pixelFlags_cr',
 'pixelFlags_bad',
 'forcedSourceId',
 'detector',
 'visit']

In [38]:
in_mem_comcam_gaia = comcam_gaia.head(100)

In [50]:
comcam_mag = in_mem_comcam_gaia.iloc[0]["lc_object"].psfMag
band = in_mem_comcam_gaia.iloc[0]["lc_object"].band
phot_rp_mean_mag_gaia = in_mem_comcam_gaia.iloc[0]["phot_rp_mean_mag_gaia"]
print(f"Band {band} psfMag comcam {comcam_mag} phot_rp_mean_mag_gaia {phot_rp_mean_mag_gaia}")

Band 0    r
Name: band, dtype: object psfMag comcam 0    17.941202
Name: psfMag, dtype: float64 phot_rp_mean_mag_gaia 17.476751


### Crossmatching bad or photometric instability??
Just from poking around at the magnitudes

TODO:
* move back into LSDB land (is reduce supported now?)
* confirm if psfMag is the column we want. As always column discoverability is tricky.
* check other bands

In [ ]:
# Outside of dataframe world, just see what the differences are.
# TODO move back into LSDB land
diffs = []
invalid_cnt = 0
magnitude_cutoff = 1
for i in range(len(in_mem_comcam_gaia)):
    row = in_mem_comcam_gaia.iloc[i]
    sep = abs(row['lc_object'].psfMag - row['phot_rp_mean_mag_gaia'])
    # Mask to only using r band (which is what phot_rp_mean_mag_gaia)
    sep = sep[row['lc_object'].band == 'r']
    if (sep > magnitude_cutoff).any():
        invalid_cnt += 1
        print(f"Oh no:( Band {band} psfMag comcam {row['lc_object'].psfMag} phot_rp_mean_mag_gaia {row['phot_rp_mean_mag_gaia']} sep {sep}")
    diffs.append(sep)
invalid_cnt
    

Oh no:( Band 0    r
Name: band, dtype: object psfMag comcam 0    17.575642
1    17.576106
Name: psfMag, dtype: float64 phot_rp_mean_mag_gaia 16.404354 sep 0    1.171288
1    1.171752
Name: psfMag, dtype: float64
Oh no:( Band 0    r
Name: band, dtype: object psfMag comcam 0    17.575642
1    17.576106
Name: psfMag, dtype: float64 phot_rp_mean_mag_gaia 16.404354 sep 0    1.171288
1    1.171752
Name: psfMag, dtype: float64
Oh no:( Band 0    r
Name: band, dtype: object psfMag comcam 0    17.575640
1    17.576109
Name: psfMag, dtype: float64 phot_rp_mean_mag_gaia 16.404354 sep 0    1.171286
1    1.171755
Name: psfMag, dtype: float64
Oh no:( Band 0    r
Name: band, dtype: object psfMag comcam 0    17.575640
1    17.576109
Name: psfMag, dtype: float64 phot_rp_mean_mag_gaia 16.404354 sep 0    1.171286
1    1.171755
Name: psfMag, dtype: float64
Oh no:( Band 0    r
Name: band, dtype: object psfMag comcam 0    17.396634
Name: psfMag, dtype: float64 phot_rp_mean_mag_gaia 14.861088 sep 0    2.53554

45

# Apply Forced Photometry
Measure standard deviation of the points using forced photometry and forced photomery on difference images. 

# Compare your measurements with the reported errors

